##Pips

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

from transformers import *


##Load Datasets

In [ ]:
path1 = '/content/drive/MyDrive/datasets/modelsum_train.csv'
path2 = '/content/drive/MyDrive/datasets/modelsum_test.csv'

In [ ]:
final_df = pd.read_csv(path1)
test_df = pd.read_csv(path2)

In [ ]:
final_df.shape, test_df.shape

((10016, 2), (1200, 2))

In [ ]:
#this is necessary because model won't run if there are NAN values. Though Training set is clean, this is for double check
final_df = final_df.dropna() 
final_df.label.value_counts()

1    5008
0    5008
Name: label, dtype: int64

In [ ]:
final_df.label.value_counts()

1    5008
0    5008
Name: label, dtype: int64

In [ ]:
final_df.head()

,label,text
0,1,বাংলাদেশের কৃষি বিশ্ববিদ্যালয়ে উপাচার্যের কার...
1,1,বাংলাদেশের প্রবাসী কল্যাণ ও বৈদেশিক কর্মসংস্থা...
2,1,বাংলাদেশের উত্তরাঞ্চলীয় জেলা মনোহরদীর মনোহরদী...
3,1,বাংলাদেশেও মামলাজট নিরসনে বিকল্প বিরোধ নিষ্পত্...
4,1,বাংলাদেশের প্রধানমন্ত্রী শেখ হাসিনার নির্দেশনা...


#MODEL TRAINING

In [ ]:
from transformers import BertTokenizer, AutoTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained('DeadBeast/mbert-base-cased-finetuned-bengali-fakenews')

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--DeadBeast--mbert-base-cased-finetuned-bengali-fakenews/snapshots/ddac3dee4e27c3ee4c08e528c7bbcb3345c90ca4/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--DeadBeast--mbert-base-cased-finetuned-bengali-fakenews/snapshots/ddac3dee4e27c3ee4c08e528c7bbcb3345c90ca4/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--DeadBeast--mbert-base-cased-finetuned-bengali-fakenews/snapshots/ddac3dee4e27c3ee4c08e528c7bbcb3345c90ca4/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--DeadBeast--mbert-base-cased-finetuned-bengali-fakenews/snapshots/ddac3dee4e27c3ee4c08e528c7bbcb3345c90ca4/tokenizer_config.json


In [ ]:
max_len = 0
max_len_itr =0

for i in range(len(final_df)):
  sent = sent = final_df['text'][i]

  # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
  input_ids = tokenizer.encode(sent, add_special_tokens=True)

  # Update the maximum sentence length.
  leng = len(input_ids)
  if(leng>max_len):
    max_len = leng
    max_len_itr = i

print('Max sentence length: ', max_len)
print('sentence: ', final_df['text'][max_len_itr])

Max sentence length:  511
sentence:  তান্দুরি চা!তান্দুরি চিকেন তো খেয়েছেন, তান্দুরি চা খেয়েছেন কখনও? ভাবছেন এটি আবার কেমন চা! ভারতের পুনেতে গিয়ে খুবই বিখ্যাত একটি খাবারের নাম বলতে বললে তান্দুরি চায়ের নাম আসবেই। নতুন স্বাদের এই চা তৈরি করতে প্রয়োজন হবে মাটির পাত্রের। মাটির পাত্রের পোড়া কিংবা স্মোকি গন্ধই এই চায়ের বিশেষত্ব। জেনে নিন কীভাবে তৈরি করবেন তান্দুরি চা। উপকরণ গরুর দুধ- ১ কাপ চা পাতা- দেড় টেবিল চামচ চিনি- স্বাদ মতো আদার টুকরা- ৩টি এলাচ- ১টি প্রস্তুত প্রণালি একটি মাটির পাত্র মিডিয়াম লো আঁচে গরম করে নিন চুলায়। ছোট কিংবা মাঝারি সাইজের পাত্র নিতে পারেন। চায়ের পরিমাণের উপর নির্ভর করবে পাত্রের আকার। চিমটার সাহায্যে ঘুরিয়ে ফিরিয়ে ভালো করে গরম করুন মাটির পাত্র। পাত্রের রঙ বদলে যাওয়া পর্যন্ত গরম করতে হবে। চুলায় মাটির পাত্র গরম দিয়ে অন্য চুলায় চা বসিয়ে দিন। প্যানে গরুর দুধ, আধা কাপ পানি, এলাচ ও আদার টুকরা দিন। বলক আসলে চা পাতা ও চিনি দিয়ে দিন। ৫ মিনিট পর ছাঁকনি দিয়ে ছেঁকে একটি বড় পাত্রে নিয়ে নিন চা। এবার চায়ে তান্দুরি ফ্লেভার নিয়ে আসার জন্য একটি বড় প্যান নিন। চুলা থেকে মাটির গরম পাত্র উঠ

In [ ]:
label_list = []
for label in final_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in final_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
print('Original: ', final_df['text'][0])
print('Token IDs:', input_ids[0])

Original:  বাংলাদেশের কৃষি বিশ্ববিদ্যালয়ে উপাচার্যের কার্যালয়ে হট্টগোলের ঘটনায় দুইজনকে সাময়িক বরখাস্ত ও ছয় জনকে শোকজ করেছে বিশ্ববিদ্যালয় প্রশাসন। বাকৃবিতে হামলাকারীদের কারণ দর্শানোর নোটিশ দেয়া হয়েছে গত ১৭ সেপ্টেম্বরের দুপুরে। এ ঘটনাকে কেন্দ্র করে   Kuniকেল কোর্টের নির্দেশ অনুযায়ী আগামী ৭ দিনের মধ্যে উত্তর দেয়ার পরামর্শ দিয়েছেন বিশ্ববিদ্যালয়ের শিক্ষার্থীরা।. এর সঙ্গে BBC 코리아 .. ( ) - এই নিয়ে , বিবিসি । (সংবাদিকদের ধারণা : 'আন্তর্জাতিক নকল প্রকাশ বন্ধ করে দেয় প্রধান পক্ষ সরকার।'  এ বিভক্তি সবসময়ই হলেও, এখনো এ বিষয়ে আবারো আলোচনা চলছে। তবে এবারের আন্দোলনে এমন তিনজনকে গ্রেপ্তার করা হচ্ছে নতুন করে দুইজন বিশ্ববিদ্যালয় শিক্ষক এবং শিক্ষকদের বিরুদ্ধে বিভিন্ন ক্ষেত্রে আন্দোলন শুরু হওয়ার পর এখন পর্যন্ত সাক্ষীর সংখ্যা ছাড়াও এসব ব্যবস্থা গ্রহণ করতে চাইছে ছাত্র অধিকার সংগঠনগুলোর একটি বড় অংশের সাথে সামাজিক যোগাযোগ মাধ্যম ভিত্তিক সূত্রগুলোয় দেখা গেছে, যেখানে এ ধরণের বিতর্ক চরমে পৌঁছানো হয়েছে একাধিক অভিযোগের কথা রয়েছে এখানকার বিশ্ববিদ্যালয় কর্তৃপক্ষ।
Token IDs: tensor([   101,  3

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.85 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

8,513 training samples
1,503 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "DeadBeast/mbert-base-cased-finetuned-bengali-fakenews", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Downloading:   0%|          | 0.00/850 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--DeadBeast--mbert-base-cased-finetuned-bengali-fakenews/snapshots/ddac3dee4e27c3ee4c08e528c7bbcb3345c90ca4/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "torch_dtype": "flo

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--DeadBeast--mbert-base-cased-finetuned-bengali-fakenews/snapshots/ddac3dee4e27c3ee4c08e528c7bbcb3345c90ca4/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at DeadBeast/mbert-base-cased-finetuned-bengali-fakenews.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            loss = outputs[0]
            logits = outputs[1]
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    533.    Elapsed: 0:01:00.
  Batch    80  of    533.    Elapsed: 0:01:59.
  Batch   120  of    533.    Elapsed: 0:03:00.
  Batch   160  of    533.    Elapsed: 0:04:00.
  Batch   200  of    533.    Elapsed: 0:05:01.
  Batch   240  of    533.    Elapsed: 0:06:02.
  Batch   280  of    533.    Elapsed: 0:07:02.
  Batch   320  of    533.    Elapsed: 0:08:02.
  Batch   360  of    533.    Elapsed: 0:09:03.
  Batch   400  of    533.    Elapsed: 0:10:03.
  Batch   440  of    533.    Elapsed: 0:11:04.
  Batch   480  of    533.    Elapsed: 0:12:04.
  Batch   520  of    533.    Elapsed: 0:13:05.

  Average training loss: 0.20
  Training epcoh took: 0:13:23

Running Validation...
  Accuracy: 0.95
  Validation Loss: 0.15
  Validation took: 0:00:50

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    533.    Elapsed: 0:01:01.
  Batch    80  of    533.    Elapsed: 0:02:01.
  Batch   120  of    533.    Elapsed: 0:03:01.
  Batch  

In [ ]:
model.save_pretrained('/content/drive/MyDrive/saved_models/deadbeast-mbert/')

Configuration saved in /content/drive/MyDrive/saved_models/deadbeast-mbert/config.json
Model weights saved in /content/drive/MyDrive/saved_models/deadbeast-mbert/pytorch_model.bin


In [ ]:
#loading models
loaded_model = BertForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/saved_models/deadbeast-mbert/",
    num_labels = 2
)

# Tell pytorch to run this model on the GPU.
loaded_model.cuda()

loading configuration file /content/drive/MyDrive/saved_models/deadbeast-mbert/config.json
Model config BertConfig {
  "_name_or_path": "DeadBeast/mbert-base-cased-finetuned-bengali-fakenews",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

#Model Testing

In [ ]:
test_df.shape

(1200, 2)

In [ ]:
test_df = test_df.dropna()
test_df.label.value_counts()

0    600
1    600
Name: label, dtype: int64

In [ ]:
test_df.tail()

,label,text
1195,1,বাংলাদেশের জনপ্রিয় কথাসাহিত্যিক মোস্তফা কামাল...
1196,1,‘জনসভা নিয়ে সরকারি দলের বিভিন্ন কথা অত্যন্ত দু...
1197,1,শয়নকক্ষ থেকে চুরি হওয়া নবজাতকের লাশ মিলল পুকুর...
1198,1,তারেক রহমানকে ফাঁসানোর ষড়যন্ত্র চলছে : মির্জা ...
1199,1,বিবিএস কেবলস লিমিটেড'র ডিলার কনফারেন্সকেবলস ম্...


In [ ]:
label_list = []
for label in test_df['label']:
  label_list.append(label)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_df['text']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        truncation = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(label_list)

# Print sentence 0, now as a list of IDs.
print('Original: ', test_df['text'][0])
print('Token IDs:', input_ids[0])

Original:  বাংলাদেশে মানবতাবিরোধী অপরাধের বিচার নিয়ে তীব্র ক্ষোভ প্রকাশ করেছেন দৈনিক মতিকণ্ঠ। মুক্তিযুদ্ধের সময় বিএনপির চেয়ারপার্সন আবদুল কাদের সিদ্দিকী এবং তাঁর পরিবারের সদস্যরা তাকে গ্রেপ্তার ছাড়ার জন্য সরকার ব্যবস্থা গ্রহণ করেছে.  ,  BBC নিউজ এর পক্ষ থেকে এ বিষয়ে তৈরি কোন তথ্য বের হচ্ছে না - যার মধ্যে এক  Kuni ( ( ) ? ' বিবিসি ’ র সঙ্গে এখনও এই দেশে মানসিকতার বিচারে যুক্ত নেই .. এ সব ক্ষেত্রেই তারা গত কাল ঢাকায় জন্ম হল । এমন কী ভাবেন জাতিসংঘের একজন সাবেক প্রধানমন্ত্রী জিয়া খানকে গুলি করেন অধ্যাপক গোলাম আযমের বিরুদ্ধে যে অভিযোগ উঠেছে, সেই অভিজ্ঞতার কথা বললেও বিবিসির সংবাদ মাধ্যমে এসব খবর পাওয়া গেছে ইস্কুলের মাষ্টার, যিনি অবৈধভাবে টিক্কার সাথে যৌন সম্পর্কে লিপ্ত ছিলেন, তাদের দেশছাড়া তিনি এখনো বন্ধ করা হয়নি ; কিন্তু পাকিস্তানী নাগরিকদের কাছে এত মানুষের মত লোকজন এখানে আবারো যুদ্ধাপরাধীদের মৃত্যুর ঘটনা তো ঠিকই ছিলো, যে তিনজনের সামাজিক যোগাযোগ
Token IDs: tensor([   101,  27470,  11199,  18601,  11737,  19910,  19668,  47356,  11128,
         16431,  56251,  13100,    937,  62210

We can see some of the results by the model here. Our model trains on half of the dataset and achieves around 0.80 in overall f1. Its likely that the model is trained longer it will achieve better performance. I might retrain it later on full data.

In [ ]:
testdataset = TensorDataset(input_ids, attention_masks, labels)

In [ ]:
test_dataloader = DataLoader(
            testdataset, # The validation samples.
            sampler = SequentialSampler(testdataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

y_pred = []
y_true = []

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        outputs = loaded_model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]

        _, prediction = torch.max(logits, dim=1)
        targets = b_labels.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()

        y_pred.extend(prediction)
        y_true.extend(targets.tolist())
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)

print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Test took: {:}".format(validation_time))

Running Testing...
  Accuracy: 0.70
  Test Loss: 1.52
  Test took: 0:00:40


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.50      0.63       600
           1       0.64      0.90      0.75       600

    accuracy                           0.70      1200
   macro avg       0.74      0.70      0.69      1200
weighted avg       0.74      0.70      0.69      1200



# Resources

* fastai paper : https://arxiv.org/pdf/2002.04688.pdf
* [BERT Fine-Tuning Tutorial with PyTorch · Chris McCormick](https://mccormickml.com/2019/07/22/BERT-fine-tuning/)
* [multilingual bert](https://huggingface.co/bert-base-multilingual-cased)
* https://github.com/cdpierse/transformers-interpret
* https://blog.dataiku.com/the-learning-rate-finder-technique-how-reliable-is-it